# Statistical Tests II
---

Solutions are provided below.  
Each example contains easier parts at the start, plus more challenging extensions.  
The extensions are useful to understand the concepts more generally, and are likely close to the difficulty of the final parts of an exam question.

---

#### Example 1

Consider flipping a coin.
There are two outcomes (heads or tails), and so this is called a "Bernoulli trial" (or binomial trial), and we can calculate the likehood of getting N number of heads with a binomial distribution.  
What happens if a large group of people all flip coins?
There are likely to be a few "lucky" people who end up with all heads or all tails, which is a form of multiple hypothesis testing.
 - Use random binomial variates to simulate flipping a coin 5 times, what distribution of heads or tails do you get?
 - Create a 1000x10 matrix (simulating 1000 people flipping a coin 10 times) and calculate the number of heads each person flips. What type of distribution does this look like and why?
 - Apply different MHT correction procedures and see how the number of remaining significant results remain. How would different correction procedures change your interpretation of the results?

**Extension**
 - Does increasing/decreasing the number of people flipping coins or the number of coins each person flips make it more or less likely to get significant hits even after p value correction?
 - What sort of conditions would lead to one method being too conservative/lenient. Is it meaningful if you have a significant hit after one correction
but not another?

---

In [ ]:
# Let's flip the coin 5 times, using the binomial distribution with a probability of 0.5.
# Assume landing on heads is "1" and landing on tails is "0".
Flips <- rbinom(5,1,.5)
cat("Flips:",Flips,"\nNumber of heads:",sum(Flips),"\n\n")


# Now let's simulate this for many people.
# Flip 10,000 coins, and split into 1,000 people (so same as each flipping 10 times).
Flips <- matrix(rbinom(10000,1,.5),nrow = 1000)

# Sum over each row, so counting how many heads each person flipped.
# Print the mean number of heads each person flipped, and a histogram.
Sums <- rowSums(Flips)
MeanHeads <- mean(Sums)
cat("Mean number of heads flipped:",MeanHeads,"\n")
hist(Sums,breaks = seq(-1,11,length.out = 12))

# How does it look per person?
# Would someone say the coin if unfair if they rolled 2 heads? 9 heads?
cat("Rolling 2 heads: p =",binom.test(2,10,.5)$p.value,"\nRolling 9 heads: p =",binom.test(9,10,.5)$p.value,"\n")

# We have to correct for the number of people "testing" the coin, in this case 1000 people.
# So the critical p-value isn't 0.05 anymore, but 0.05/1000 = 5e-5.

In [ ]:
# Let's push it a bit further, consider N flips and B people (so N/B flips per person)

N = 10000
B = 500
cat("Number of flips per person: ",N/B,"\n")

# Change the seed used to generate random numbers, so we can *reproducibly* generate random datasets.
# It is a complicated topic, but don't fall into the trap that similar seeds produce similar results.
# Try commenting one value out and swapping the other in and see what happens.

set.seed(6)
#set.seed(5)

# Generate per-person p values for a binomial test.
# `sapply` applies the function given in the second argument to each element of data in the first argument.
# This gives us the raw p values that each person calculated for their set of N/B flips.

RawP <- sapply(sort(rowSums(matrix(rbinom(N,1,.5),nrow = B))), function(x) binom.test(x, N/B, .5)$p.value)
cat("Number of \"significant\" tests without correction:",sum(RawP<0.05),"\n")               

# Now we can correct the p values using two different methods.
# Note that this adjusts the p values upwards, which is essentially equal to but more obvious than adjusting the significance threshold downwards.

BP <- p.adjust(RawP,"bonferroni")
BHP <- p.adjust(RawP,"BH")
cat("Number of \"significant\" tests with Bonferroni correction:",sum(BP<0.05),"\n")       
cat("Number of \"significant\" tests with Benjamini-Hochberg correction:",sum(BHP<0.05),"\n")       
               
# Plot the raw p values on the x-axis, and the two methods of correction on the y-axis.
# The significance threshold is still p < 0.05, marked by the line.

plot(RawP,BP,col=rgb(0,0,1),ylim=c(0,1),xlab="Raw p",ylab="Adjusted p")
par(new = TRUE)
plot(RawP,BHP,col=rgb(1,0,0),ylim=c(0,1),xlab="",ylab="")
abline(0.05,0)
legend("right",c("Bonferroni","Benjamini-Hochberg"),fill=c(rgb(0,0,1),rgb(1,0,0)))

In [9]:
## Example 1 extension

# Does increasing/decreasing N or B make it more or less likely to get significant hits even after p value correction?

# Repeat with a very distinct set of values
N = 10000
B = 1000
cat("Number of flips per person: ",N/B,"\n")
RawP <- sapply(sort(rowSums(matrix(rbinom(N,1,.5),nrow = B))), function(x) binom.test(x, N/B, .5)$p.value)
BP <- p.adjust(RawP,"bonferroni")
cat("Number of \"significant\" tests without correction:",sum(RawP<0.05),"\nNumber of \"significant\" tests with Bonferroni correction:",sum(BP<0.05),"\n")               

N = 10000
B = 10
cat("Number of flips per person: ",N/B,"\n")
RawP <- sapply(sort(rowSums(matrix(rbinom(N,1,.5),nrow = B))), function(x) binom.test(x, N/B, .5)$p.value)
BP <- p.adjust(RawP,"bonferroni")
cat("Number of \"significant\" tests without correction:",sum(RawP<0.05),"\nNumber of \"significant\" tests with Bonferroni correction:",sum(BP<0.05),"\n")               


# The corrections are very robust, and so in general will never report any significant hits.
# However, increasing N/B (so more flips per person) can result in a person getting a very signficant outlier, which may be significant after correction.
# Similarly, decreasing N/B (so more people flipping less coins) means the correction factor is higher, so even less likely for a signficant result after correction.

 
               
# What sort of conditions would lead to one method being too conservative/lenient. Is it meaningful if you have a significant hit after one correction
# but not another?

# If your probabilities are very unevenly distibuted (so a few highly signficiant values and a few non-signifant values), then the Bonferroni correction will be much stricter than B-H.
# They are both valid choices, so there is not any meaningful conclusion if p-values are significant after one correction but not the other. 
# However, it is important not to change your method because you didn't like the outcome (p-value hacking!).

Number of flips per person:  10 
Number of "significant" tests without correction: 17 
Number of "significant" tests with Bonferroni correction: 0 
Number of flips per person:  2000 
Number of "significant" tests without correction: 0 
Number of "significant" tests with Bonferroni correction: 0 


---
#### Example 2

We will examine the data in Frossard_2019 again, looking at statistical results across different variables.
 - Examine the variance of the *DW.grain.kg.ha* and *SPAD.afternoon* variables, are they similar? Are they statistically significantly similar?
 - Comparing the Bockris and Titlis varieties, is there a statistically significant difference in variances?
 - Test if the amount of fertilizer used has an impact on the *harvest.index*.
 - What happens if we swap the order of variables around. This didn't matter for correlation analyses, will it here?

**Extension**
 - Test if *DW.grain.kg.ha* and *SPAD.afternoon* are normally distributed variables.
 - Test for any significant differences between the above variables.
 - Linear regression (covered soon) will also produce an F-statistic, can we calculate the sample size needed for this to be significant?

---

In [ ]:
# Load in data used in the previous lecture
crop_data <- read.table('Frossard_2019.csv',header=TRUE,sep=",")
head(crop_data)

# Let's look at the variance for two of the variables.
var(crop_data$DW.grain.kg.ha)
var(crop_data$SPAD.afternoon)

# We can use an F-test to see if they have equal variances.
var.test(crop_data$DW.grain.kg.ha,crop_data$SPAD.afternoon)

In [ ]:
# Slightly more advanced, but we can compare the variances between two groups (the two varieties) for specific variables.
# Here, let's look at the "DW.grain.kg.ha" quantity, and see if there is a difference in variance for the two varieties.
# We can do that manually by separating out the data and testing (using the more robust Bartlett test), or a more advanced notation using "~".
bockris <- crop_data[crop_data$variety == "Bockris",]
titlis <- crop_data[crop_data$variety == "Titlis",]
bartlett.test(list(bockris$DW.grain.kg.ha,titlis$DW.grain.kg.ha))

# And this should be equal (but easier to write!)
bartlett.test(DW.grain.kg.ha ~ variety, data = crop_data)
var.test(DW.grain.kg.ha ~ variety, data = crop_data)

In [ ]:
# Kruskal-Wallis is the appropriate test to use
kruskal.test(harvest.index ~ fertilizer, data = crop_data)

# The order-response variables **matters** a lot.
# If you swap the order, it is no longer the same analysis (as it was in correlation and many other statistical tests).
kruskal.test(fertilizer ~ harvest.index, data = crop_data)

In [ ]:
## Example 2 extension

# Consider another set of variables, are they normal?
shapiro.test(crop_data$DW.grain.kg.ha)
shapiro.test(crop_data$SPAD.afternoon)

# The data does approximately satisfy the requirements for an F-test, so the nonparametric version is less powerful than the parametric version covered soon (ANOVA).
kruskal.test(DW.grain.kg.ha ~ SPAD.afternoon, data = crop_data)

# This is jumping ahead to a future lecture, but this will calculate the linear regression and plot the fit.
# This uses F-tests as we covered in the first Statistical Tests lecture, but here is not a large enough sample size to be significant.
regression <- lm(DW.grain.kg.ha ~ SPAD.afternoon, data=crop_data)
summary(regression)
plot(DW.grain.kg.ha ~ SPAD.afternoon, data = crop_data)
abline(regression,col="red",lw=4)

# Given the F statistic calculated from the regression,  how many samples would we need (assuming they follow the same pattern as the existing data) to be significant?
# We can calculate this based on the F-distribution and look for the critical value we need (p=0.05)
my.F <- summary(regression)$fstatistic[1]

# Creates a range of sample sizes we want to calculate the significance for.
my.n <- 18:1000

# Calculate significance for each n and plot.
my.p <- pf(my.F, 1, my.n-2, lower.tail=F)
plot(my.p, my.n, type="l")
abline(v=0.05, col="red")

---
#### Example 3

We can also examine the PlantGrowth dataset based on observations for growth using different treatments.
 - Make a boxplot of the *weight* against each of the *group* variables.
 - Test if there are any significant differences present.
 - Use a nonparametric posthoc test to find which groups result in different growth rates.

**Extension**
 - The default pairwise test correction uses the "Holm" procedure. How do other options ("none","hochberg", "hommel", "bonferroni", "BH", "BY") compare?

---

In [ ]:
# Load a default dataset for the weight of plants after 3 different types of treatments.
# Plot the weights as boxplots to visualise potential group differences.
boxplot(weight ~ group, data = PlantGrowth, frame = FALSE,
        col = c("#999999", "#E69F00", "#56B4E9"))

# We want to test if there is any significant differences between the three groups.
# We will use the Kruskal-Wallis test.
kruskal.test(PlantGrowth$weight, PlantGrowth$group)


# However, this just shows that there is a significant difference, but it is not clear between which groups.
# We can do pairwise tests (e.g. "ctrl" and "trt1", "ctrl" and "trt2", etc.) to see which are significant.
# Be careful, as we are now testing multiple hypotheses and so should correct accordingly.
pairwise.wilcox.test(PlantGrowth$weight, PlantGrowth$group,   p.adjust.method = "holm",exact=F)

In [ ]:
## Example 3 extension

pairwise.wilcox.test(PlantGrowth$weight, PlantGrowth$group,   p.adjust.method = "none",exact=F)

---
#### Example 4

We can also examine the iris dataset based on observations for flower growth.
 - Create a boxplot using the builtin "iris" dataset, plotting *Sepal.width* against *Species*.
 - Test if any species have a statistically different sepal width.
   - Use a posthoc test to find out which species may be significantly different.
 - Use the `matrixTests` package to test for differences across multiple variables at once, and confirm it matches the results from before.

**Extension**
 - Perform posthoc tests for the multiple Kruskal-Wallis tests conducted when using the `matrixTests` package in the previous part.

---

In [ ]:
# We can also look at a different dataset.
# Use the "iris" default dataset for flowers.
head(iris)

# Here we will plot the sepal width for the three different species.
boxplot(Sepal.Width ~ Species, data = iris, frame = FALSE,
        col = c("#999999", "#E69F00", "#56B4E9"))

# However, there are other variables, so we can try and plot this as an "all-versus-all" pair plot.
# First we should convert the numeric data to a matrix, and also turn the species names into numeric values to make plotting easier.
ma <- as.matrix(iris[, 1:4])
speciesID <- as.numeric(iris$Species)
pairs(ma, col = rainbow(3)[speciesID])

In [ ]:
# Let's test the significance of sepal length for the different species.
kruskal.test(iris$Sepal.Width, iris$Species)

# And again do a pairwise test to see which groups are significant.
pairwise.wilcox.test(iris$Sepal.Width, iris$Species,   p.adjust.method = "BH",exact=F)

In [ ]:
# Install and load a package called matrixTests we will use for simplicity.
install.packages("matrixTests")
library("matrixTests")

In [ ]:
# We can do a column-wise test for all the variables.
col_kruskalwallis(ma, speciesID)

# Compare to doing each test manually, and confirm identical results.
# Note the p-values that are too small get reported as < 2.2e-16, because the computer loses accuracy for numbers smaller than this.
# To get the exact number back, we will add $p.value immediately to get the full accuracy value.
kruskal.test(iris$Sepal.Length, iris$Species)$p.value
kruskal.test(iris$Sepal.Width, iris$Species)$p.value
kruskal.test(iris$Petal.Length, iris$Species)$p.value
kruskal.test(iris$Petal.Width, iris$Species)$p.value

In [ ]:
## Example 4 extension

# There is no obvious way to do multiple multiple hypothesis testing, so we can do this manually for the four tested variables.
pairwise.wilcox.test(iris$Sepal.Length, iris$Species,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(iris$Sepal.Width, iris$Species,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(iris$Petal.Length, iris$Species,   p.adjust.method = "BH",exact=F)
pairwise.wilcox.test(iris$Petal.Width, iris$Species,   p.adjust.method = "BH",exact=F)

---
## Real world example

This is a slightly more realistic case, where we are examining some [actual data](https://opendata.swiss/en/dataset/rinder-geburten-nach-kantonen).  
The goal here is to manipulate the data into a useful shape, and then assess statistical significance.  
These examples are harder than you would encounter in an exam and use more advanced data analysis techniques, but are useful to try.

This data is the number of cattle born each month for each canton in Switzerland
 - Similar to the exercise in *Statistical Tests I*, aggregate the data over each month to make annual totals.
 - Subset out some cantons and look if the annual number of births is correlated between them.
 - Reshape the data to be "long" rather than "wide" ("melting" the different columns for each canton to become a variable in each row).
 - Create a boxplot for a subset of the cantons and then conduct a statistical test to check if any canton has a significant difference.
   - Conduct posthoc tests to find which cantons are the significantly different ones (taking care to correct for MHT). 

---

In [ ]:
# Read in the semi-colon separated data, skipping the first line which is a comment.
births <- read.csv('cattle-birthCanton.csv',sep=';',skip=1)

# We want to sum over *all* columns by aggregating over year, so we can use the ~ syntax over "." (all coloumns). 
grouped_births <- aggregate(. ~ Year,data=births, FUN=sum)
grouped_births



# and we can test for correlations between the cantons, here are two examples.
cor.test( grouped_births$AG, grouped_births$VD)
cor.test( grouped_births$BE, grouped_births$VS)

# We want to test if there is any significant differences between the three groups.
# We will use the Kruskal-Wallis test.

# We'll reshape the dataframe ("melt" it) so the cantons change from being columns to being row values
melted_births <- reshape2::melt(grouped_births, id.vars = c("Year", "Month"),variable.name = "Canton", value.name = "Births")

# We'll look at a subset of four cantons and plot the number of births of the different years
melted_births_subset1 <- melted_births[melted_births$Canton %in% c('AG','VD','BE','VS'),]
plot(Births ~ Year, data = melted_births_subset1,col=factor(melted_births_subset1$Canton),frame = FALSE)

# We'll take another subset of cantons 
melted_births_subset2 <- melted_births[melted_births$Canton %in% c('GR','TG','ZH','SZ','SO','UR','TI'),]

# We need to convert the column type from "fct" (function) to "str" (string), which will remove all the cantons we don't care about above.
# You should try redoing the boxplot after commenting out these two lines and seeing what changes.
i <- sapply(melted_births_subset2, is.factor)
melted_births_subset2[i] <- lapply(melted_births_subset2[i], as.character)


boxplot(Births ~ Canton, data = melted_births_subset2, frame = FALSE)

# We will use the Kruskal-Wallis test to test if there is any significant differences between the cantons.
kruskal.test(Births ~ Canton, data = melted_births_subset2)

# However, this just shows that there is *a* significant difference, but it is not clear between which groups.
# We can do pairwise tests (e.g. SZ and ZH, SZ and SO, etc.) to see which are significant.
# Be careful, as we are now testing multiple hypotheses and so we need to correct accordingly.
pairwise.t.test(melted_births_subset2$Births, melted_births_subset2$Canton, p.adjust.method = "bonf",paired=TRUE)